In [1]:
import numpy as np
import pandas as pd
import os
import time
os.getcwd()
os.chdir('C:\\Users\\Wetauzer\\Desktop\\Data')

In [ ]:
start_time = time.time()
tx_df = pd.read_csv('Z://10000000to10999999_NormalTransaction.csv', nrows=100000)
tx_df = tx_df.loc[:, tx_df.columns.intersection(['blockNumber','transactionHash', 'gasPrice'])]
tx_df = tx_df.rename(columns = {'blockNumber':'block','transactionHash':'hash', 'gasPrice':'gas_price'})
tx_df.gas_price = tx_df.gas_price/1e9 # convert the prices to gwei
tx_df = tx_df[tx_df['gas_price']!=0] # Getting rid of zero gas price transactions
block_df = pd.DataFrame(tx_df["block"].copy().unique()).rename(columns = {0: 'block'})

Below is the function for the "price oracle". It returns a dataframe of all transactions with quantile data from the last N blocks at the time the transaction was exectuted. You can change the quantiles but by default it returns the min/max and 50th, 75th, and 90th percentile. The window tells it how many of the previous blocks' transactions to calculate the summary based statistics on. By default, N = 10 blocks, so the output will be a dataframe starting with transactions from the Nth block with summary statistics from the first N blocks.

In [ ]:
def oracle_prices(transaction_df, block_df, low = 50, medium = 75, high = 90, window = 10):
    """Gives min/max and 3 quantile values over the last # of blocks (by default 10)"""
    blx_in = block_df['block']
    blx_out = []
    p1 = []
    p2 = []
    p3 = []
    p4 = []
    p5 = []
    for blk in blx_in:
        if blk - window > blx_in.min():
            idx = pd.Index(list(blx_in)).get_loc(blk)
            blk_win = blx_in[idx - window + 1 : idx + 1]
            blx_out.append(blk)
            p1.append(tx_df[tx_df.block.isin(blk_win)].gas_price.min())
            p2.append(tx_df[tx_df.block.isin(blk_win)].gas_price.quantile(low/100))
            p3.append(tx_df[tx_df.block.isin(blk_win)].gas_price.quantile(medium/100))
            p4.append(tx_df[tx_df.block.isin(blk_win)].gas_price.quantile(high/100))
            p5.append(tx_df[tx_df.block.isin(blk_win)].gas_price.max())
    dictionary = {'block':blx_out, 'minimum':p1, 'low':p2, 'average':p3, 'fast':p4, 'maximum':p5}
    oracle_df = pd.DataFrame(dictionary)
    tx_oracle_df = pd.merge(tx_df, oracle_df, how="left", on=["block"])
    tx_oracle_df = tx_oracle_df[tx_oracle_df.block >= blx_in[window]]
    return tx_oracle_df
oracle_df = oracle_prices(tx_df, block_df)
oracle_df

Below is the assign values function. It's really basic. It just assigns a value using linear interpolation and what the gas price is compared to the summary statistics. It needs work but it's a start.

In [ ]:
def assign_value(tx):
    gp = tx[2]
    p1 = tx[3]
    p2 = tx[4]
    p3 = tx[5]
    p4 = tx[6]
    p5 = tx[7]
    valstd = [0, .5 , 1, 1.5, 2]
    value = np.interp(gp, [p1, p2, p3, p4, p5], valstd)
    return value

In [ ]:
oracle_df['value'] = oracle_df.apply(assign_value, axis=1)
oracle_df

In [ ]:
oracle_df.value.plot.hist()

In [ ]:
print("Total time running:", time.time() - start_time)